# Final Project: Data Engineering Workflow for European Online Retail Company
This notebook implements the end-to-end data engineering pipeline based on the scenario provided in the hands-on lab.

## Scenario Overview
As a data engineer, you're tasked with designing and implementing a data pipeline using the Online Retail dataset. The company does not want cloud-based solutions and requires an SQL-based central data repository accessible by employees from multiple countries.

## Step 1: Data Architecture Design
The data architecture involves:
- **Data Sources**: Sales transactions, customer profiles, inventory data.
- **Data Collection**: On-premises ETL process from CSV to SQL.
- **Data Transformation**: Data cleaning, formatting, and normalization.
- **Data Modeling**: SQL schema with primary/foreign keys.
- **Data Access**: Employee roles with read/write control.
- **Analysis Tools**: SQL queries, visualization via BI tools (e.g., Tableau).
- **Backup & Governance**: Daily backups, access logging, GDPR compliance.

In [2]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 10.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Step 2: Load Libraries and Dataset
import pandas as pd
df = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# Step 3: Clean and Transform Data
df.dropna(subset=['CustomerID'], inplace=True)
df = df[df['Quantity'] > 0]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## Step 4: Database Schema Design (ERD)
- **Customers**(`CustomerID`, `Country`)
- **Products**(`StockCode`, `Description`)
- **Invoices**(`InvoiceNo`, `InvoiceDate`, `CustomerID`)
- **InvoiceItems**(`InvoiceNo`, `StockCode`, `Quantity`, `UnitPrice`, `TotalPrice`)

In [5]:
# Step 5: Connect to SQLite and Load Data
import sqlite3
conn = sqlite3.connect('retail.db')
df.to_sql('retail_data', conn, if_exists='replace', index=False)
conn.commit()

In [6]:
# Step 6: Sample SQL Queries
query = '''
SELECT Country, COUNT(DISTINCT CustomerID) AS unique_customers
FROM retail_data
GROUP BY Country
ORDER BY unique_customers DESC
LIMIT 10
'''
pd.read_sql_query(query, conn)

,Country,unique_customers
0,United Kingdom,3921
1,Germany,94
2,France,87
3,Spain,30
4,Belgium,25
5,Switzerland,21
6,Portugal,19
7,Italy,14
8,Finland,12
9,Austria,11


## Step 7: Data Analysis and Mining
- Identify top countries by revenue
- Detect anomalies (negative quantities, high returns)
- Segment customers by RFM analysis

In [7]:
# Step 8: Top Revenue by Country
query_revenue = '''
SELECT Country, ROUND(SUM(TotalPrice), 2) AS Revenue
FROM retail_data
GROUP BY Country
ORDER BY Revenue DESC
LIMIT 10
'''
pd.read_sql_query(query_revenue, conn)

,Country,Revenue
0,United Kingdom,7308391.55
1,Netherlands,285446.34
2,EIRE,265545.90
3,Germany,228867.14
4,France,209024.05
5,Australia,138521.31
6,Spain,61577.11
7,Switzerland,56443.95
8,Belgium,41196.34
9,Sweden,38378.33
